# RAG Architecture

Two main components:
-Indexing
-Retrieval and generation

## Indexing
1. Load the data: this is done with DocumentLoaders
2. Split the data into smaller chunks
3. Store: you need somewhere to store and index your splits, this can be done using a VectorStore and Embeddings

## Retrieval and generation
1. Retrieve: Given the ser input relevant splits are retrieved from storage using a retriever
2. Generate: a chatmodel/llm producs the answer using a prompt that includes the question and the retrieved data

First make sure you have the needed libraries
```bash
pip install ibm-watsonx-ai==0.2.6 langchain==0.1.16 langchain-ibm==0.1.4 transformers==4.41.2 huggingface-hub==0.23.4 sentence-transformers==2.5.1 chromadb wget==3.2 --upgrade torch --index-url https://download.pytorch.org/whl/cpu
```

In [4]:
def warn(*args, **kwargs):
    pass
from pathlib import Path
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

import os, sys, textwrap
import google.generativeai as gen
from dotenv import load_dotenv
# Core libs we installed
import torch, transformers, chromadb, tokenizers, numpy as np
import langchain
from pathlib import Path


import wget

load_dotenv()
DATA_DIR   = Path("./data")        # Put your .txt/.md here (we'll add PDF later)
DB_DIR     = Path("./chroma_db")   # On-disk Chroma store
EMBED_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
CHUNK_SIZE, CHUNK_OVERLAP = 800, 120

DATA_DIR.mkdir(parents=True, exist_ok=True)
DB_DIR.mkdir(parents=True, exist_ok=True)

print("\nDATA_DIR:", DATA_DIR.resolve())
print("DB_DIR:", DB_DIR.resolve())
print("EMBED_MODEL:", EMBED_MODEL)
print(f"CHUNK_SIZE={CHUNK_SIZE}, CHUNK_OVERLAP={CHUNK_OVERLAP}")

# --- Gemini API key + model selection ---
api_key = os.getenv("GOOGLE_API_KEY")
gen.configure(api_key=api_key)



# Quick health check: minimal ping
pong = gen.GenerativeModel("gemini-2.0-flash").generate_content("Say 'pong'.").text.strip()
print("\nGemini ping:", pong)


DATA_DIR: C:\Users\Jose\Documents\GitHub\langchain_labs\proj1_lanchain_llm\data
DB_DIR: C:\Users\Jose\Documents\GitHub\langchain_labs\proj1_lanchain_llm\chroma_db
EMBED_MODEL: sentence-transformers/all-MiniLM-L6-v2
CHUNK_SIZE=800, CHUNK_OVERLAP=120

Gemini ping: Pong.


In [5]:
filename = 'companyPolicies.txt'
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/6JDbUb_L3egv_eOkouY71A.txt'

# Use wget to download the file
wget.download(url, out="data/"+filename)
print('file downloaded')

file downloaded


In [6]:

filename = 'toSummarize.txt'
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/XVnuuEg94sAE4S_xAsGxBA.txt'

# Use wget to download the file
wget.download(url, out="data/"+filename)
print('file downloaded')

file downloaded


## Upload docs and Split

In [ ]:
from pathlib import Path
from typing import List
from langchain.schema import Document
from langchain.document_loaders import DirectoryLoader, TextLoader
from  langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

loader = TextLoader(filename)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
print(len(texts))

ModuleNotFoundError: No module named 'langchain_community'